# ImkLib2 Basic Usage

imklib2 is currently not available through remote maven repositories. To install into your local maven repository, run
```
./gradlew build publishToMavenLocal
```
in the project's root directory.

In [1]:
// set up dependencies

// use local repo for now; not deployed to remote maven repo yet
@file:Repository("*mavenLocal")
// requires installation into local maven repository (./gradlew build publishToMavenLocal)
@file:DependsOn("net.imglib2:imklib2:0.1.0-SNAPSHOT")

Simply import all names from the `net.imglib2.imklib` package to get started:

In [2]:
import net.imglib2.imklib.*

## Generate Data

Create ImgLib2 `ArrayImg`s with the `imgs` object. The images are backed by Java primitive type arrays. Images can be created
 1. with dimensions only (populate with default primitive type array value, usually `0`)
 2. with dimensions and initializer that consumes linear index and type at voxel
 3. with dimensions and initializer that produces a primitive type value for linear index

Note: Use `flatStringRepresentation` only for small images.

In [9]:
// dims only (1)
println(imgs.booleans(1L, 2L, 3L).flatStringRepresentation)
// dims with initializer (2)
println(imgs.argbs(1L, 2L, 3L) { i, argb -> argb.set((i+5) * 100) }.flatStringRepresentation)
// dims with initializer (3)
println(imgs.unsignedLongs(1L, 2L, 3L) { _ -> Long.MIN_VALUE }.flatStringRepresentation)
println(imgs.floats(1L, 2L, 3L) { i -> i.toFloat() + 0.5f }.flatStringRepresentation)
println(imgs.doubles(1L, 2L, 3L) { i -> i.toDouble() + 1.0 }.flatStringRepresentation)

ArrayImg [1x2x3]: [false ,false ,false ,false ,false ,false]
ArrayImg [1x2x3]: [(r=0,g=1,b=244,a=0) ,(r=0,g=2,b=88,a=0) ,(r=0,g=2,b=188,a=0) ,(r=0,g=3,b=32,a=0) ,(r=0,g=3,b=132,a=0) ,(r=0,g=3,b=232,a=0)]
ArrayImg [1x2x3]: [9223372036854775808 ,9223372036854775808 ,9223372036854775808 ,9223372036854775808 ,9223372036854775808 ,9223372036854775808]
ArrayImg [1x2x3]: [0.5 ,1.5 ,2.5 ,3.5 ,4.5 ,5.5]
ArrayImg [1x2x3]: [1.0 ,2.0 ,3.0 ,4.0 ,5.0 ,6.0]


Virtual images can be created without any data allocation. The values are generated on request. If no interval is specified, the domain of the image will be unbounded

In [10]:
val dims = longArrayOf(1L, 2L, 3L)
println(zeros(3))
println(zeros(dims.interval).flatStringRepresentation)

net.imglib2.util.ConstantUtils$1@21f654a9
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0]


You can restrict the bounds of any image (bound or unbound) to any interval (of the same dimensionality) with the bracket operator `[]`. Note that, for bounded images, the new bounds are not checked for consistency with existing bounds.

In [11]:
println(ones(3))
println(ones(3)[dims.interval].flatStringRepresentation)
println(ones(dims.interval).flatStringRepresentation)
println(ones(longArrayOf(1, 1, 1).interval).flatStringRepresentation)

net.imglib2.util.ConstantUtils$1@5296f15c
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [1.0 ,1.0 ,1.0 ,1.0 ,1.0 ,1.0]
IntervalView [(0, 0, 0) -- (0, 1, 2) = 1x2x3]: [1.0 ,1.0 ,1.0 ,1.0 ,1.0 ,1.0]
IntervalView [(0, 0, 0) -- (0, 0, 0) = 1x1x1]: [1.0]


For the inverse operation, bounded images can be extended with various strategies. In this example, the extended images are bounded to larger bounds than the original image to demonstrate the extension. One-dimensional images are used because the visualization with flat Strings will be more meaningful.

In [34]:
val dims = longArrayOf(5)
val extended = longArrayOf(-3, 7).intervalMinMax
val img = imgs.doubles(*dims) { i -> 3.5 - abs(2.0 - i) }
println("Original image:             ${img.flatStringRepresentation}")
println("Extend with zeros:          ${img.extendZero()[extended].flatStringRepresentation}")
println("Extend with value:          ${img.extendValue(3.14)[extended].flatStringRepresentation}")
println("Extend with border value:   ${img.extendBorder()[extended].flatStringRepresentation}")
println("Extend mirror single:       ${img.extendMirrorSingle()[extended].flatStringRepresentation}")
println("Extend mirror double:       ${img.extendMirrorDouble()[extended].flatStringRepresentation}")

Original image:             ArrayImg [5]: [1.5 ,2.5 ,3.5 ,2.5 ,1.5]
Extend with zeros:          IntervalView [(-3) -- (7) = 11]: [0.0 ,0.0 ,0.0 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,0.0 ,0.0 ,0.0]
Extend with value:          IntervalView [(-3) -- (7) = 11]: [3.14 ,3.14 ,3.14 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,3.14 ,3.14 ,3.14]
Extend with border value:   IntervalView [(-3) -- (7) = 11]: [1.5 ,1.5 ,1.5 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,1.5 ,1.5 ,1.5]
Extend mirror single:       IntervalView [(-3) -- (7) = 11]: [2.5 ,3.5 ,2.5 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,2.5 ,3.5 ,2.5]
Extend mirror double:       IntervalView [(-3) -- (7) = 11]: [3.5 ,2.5 ,1.5 ,1.5 ,2.5 ,3.5 ,2.5 ,1.5 ,1.5 ,2.5 ,3.5]
